In [1]:
import networkx as nx
import numpy as np
from scipy.optimize import linprog
import pandas as pd

Below is Aashay's code for solving bandwidth allocation using linear programming.

In [2]:
m = nx.MultiGraph()

m.add_node("A", port_capacity=100)
m.add_node("B", port_capacity=400)
m.add_node("C", port_capacity=400)
m.add_node("D", port_capacity=200)

m.add_edge("A", "B", priority=3)
m.add_edge("A", "C", priority=5)
m.add_edge("B", "C", priority=1)
m.add_edge("B", "C", priority=1)
m.add_edge("B", "D", priority=5)
m.add_edge("C", "D", priority=5)

0

In [3]:
g = nx.Graph()

g.add_nodes_from(m.nodes(data=True))
# sum priorites of all edges sharing the same nodes in multigraph and add to graph
for u, v, data in m.edges(data=True):
    priority = data['priority']
    # If the edge already exists in the simple graph, sum the weights
    if g.has_edge(u, v):
        g[u][v]['priority'] += priority
    else:
        g.add_edge(u, v, priority=priority)


In [4]:
print(g.edges(data=True))

[('A', 'B', {'priority': 3}), ('A', 'C', {'priority': 5}), ('B', 'C', {'priority': 2}), ('B', 'D', {'priority': 5}), ('C', 'D', {'priority': 5})]


In [5]:
nodes = list(g.nodes)
node_index = {node: i for i, node in enumerate(nodes)}

# Step 3: List the edges and assign an index to each
edges = list(g.edges(data=True))
edge_index = {tuple(edge[:2]): i for i, edge in enumerate(edges)}

# Step 4: Initialize the incidence matrix
n_nodes = len(nodes)
n_edges = len(edges)
A = np.zeros((n_nodes, n_edges))

# Step 5: Populate the incidence matrix
for edge, data in edge_index.items():
    u, v = edge
    priority = g[u][v]['priority']
    i = node_index[u]
    j = node_index[v]
    
    A[i, data] = priority  # Outgoing edge from node i
    A[j, data] = priority  # Incoming edge to node j

In [6]:
A

array([[3., 5., 0., 0., 0.],
       [3., 0., 2., 5., 0.],
       [0., 5., 2., 0., 5.],
       [0., 0., 0., 5., 5.]])

In [7]:
edge_index

{('A', 'B'): 0, ('A', 'C'): 1, ('B', 'C'): 2, ('B', 'D'): 3, ('C', 'D'): 4}

In [8]:
b = np.fromiter(nx.get_node_attributes(g, "port_capacity").values(), dtype=float)

In [9]:
b

array([100., 400., 400., 200.])

In [10]:
success = True
lower_bound = 0
x_arr = []

while success:
    optim = linprog(c=-np.ones(n_edges), A_eq=A, b_eq=b, bounds=(lower_bound,None))
    success = optim.success
    if success:
        x_arr.append(optim.x)
    lower_bound += 1
#print(x_arr)
x = x_arr[[min(x) for x in x_arr].index(max(min(x) for x in x_arr))]
print(x)

[ 13.33333333  12.         125.          22.          18.        ]


In [11]:
x * [3,5,2,5,5]

array([ 40.,  60., 250., 110.,  90.])

Below is an attempt at solving the bandwidth allocation problem through weighted fair queueing. While I do get a solution, the bandwidth assigned exceeds capacity. There might be a way to add constraints, but for now it is what it is.

In [12]:
#Testing weighted fair queueing
#Capacities is the capacities of each node
#Priorities is the incidence matrix for the graph
def weighted_fair_queueing(capacities, priorities):
    
    n = len(capacities)
    
    # Initialize the allocation matrix X
    X = np.zeros((n, n))
    
    # Setup equality constraints for the relative priorities
    # Ex: Node A has edge (A,B) with priority 5, and edge (A,D) with priority 3. 
    # Ratio is preserved in equality in the form of [5 3 0 0 ...] and appended to list
    # This is equivalent to 5x + 3y = 0, meaning capacity should be full
    equations = []
    for i in range(n):
        for j in range(n):
            for k in range(n):
                #Only gets edges that are positive priority and not itself
                if j != k and priorities[i][j] > 0 and priorities[i][k] > 0:
                    #Creates an equation list that is empty
                    equation = np.zeros(n*n)
                    equation[i*n + j] = priorities[i][k]
                    equation[i*n + k] = -priorities[i][j]
                    equations.append(equation)
                    
    equations = np.array(equations)
    
    # Makes sure that no node exceeds its capacity
    # Represented as a 1 if the node has a connection there
    # So that the sum of used capacities is <= full capacity
    capacity_constraints = []
    for i in range(n):
        capacity_equation = np.zeros(n*n)
        for j in range(n):
            capacity_equation[i*n + j] = 1
        capacity_constraints.append(capacity_equation)
    
    capacity_constraints = np.array(capacity_constraints)
    equations = np.vstack([equations, capacity_constraints])
    
    # Solve for the allocations
    b = np.concatenate([np.zeros(len(equations) - n), capacities])
    #Solves using least squares regression
    X_vector = np.linalg.lstsq(equations, b, rcond=None)[0]
    X = X_vector.reshape((n, n))
    
    return X

In [13]:
w = nx.MultiGraph()
w.add_node("A", port_capacity=100)
w.add_node("B", port_capacity=400)
w.add_node("C", port_capacity=400)
w.add_node("D", port_capacity=200)

w.add_edge("A", "B", priority=3)
w.add_edge("A", "C", priority=5)
w.add_edge("B", "C", priority=1)
w.add_edge("B", "C", priority=1)
w.add_edge("B", "D", priority=5)
w.add_edge("C", "D", priority=5)
w.add_edge("A", "D", priority=4)

0

In [14]:
#Testing implementation of weighted fair queuing
test = nx.MultiGraph()

test.add_nodes_from(w.nodes(data=True))
# sum priorites of all edges sharing the same nodes in multigraph and add to graph
for u, v, data in w.edges(data=True):
    priority = data['priority']
    # If the edge already exists in the simple graph, sum the weights
    if test.has_edge(u, v):
        test[u][v][0]['priority'] += priority
    else:
        test.add_edge(u, v, priority=priority)

print(test.edges(data=True))

[('A', 'B', {'priority': 3}), ('A', 'C', {'priority': 5}), ('A', 'D', {'priority': 4}), ('B', 'C', {'priority': 2}), ('B', 'D', {'priority': 5}), ('C', 'D', {'priority': 5})]


In [15]:
# Create a list of nodes and a node index mapping
test_nodes = list(test.nodes)
test_node_index = {node: i for i, node in enumerate(test_nodes)}
print(node_index)

# List the edges and assign an index to each
test_edges = list(test.edges(data=True))
test_edge_index = {tuple(edge[:2]): i for i, edge in enumerate(test_edges)}
print(test_edge_index)

# Initialize the incidence matrix
test_n_nodes = len(test_nodes)
test_n_edges = len(test_edges)
test_A = np.zeros((test_n_nodes, test_n_edges))

# Populate the incidence matrix
for edge, index in test_edge_index.items():
    u, v = edge
    priority = test[u][v][0]['priority']  # Access the 'priority' attribute correctly
    i = test_node_index[u]
    j = test_node_index[v]
    
    test_A[i, index] = priority  # Outgoing edge from node i
    test_A[j, index] = priority  # Incoming edge to node j

print(test_A)

{'A': 0, 'B': 1, 'C': 2, 'D': 3}
{('A', 'B'): 0, ('A', 'C'): 1, ('A', 'D'): 2, ('B', 'C'): 3, ('B', 'D'): 4, ('C', 'D'): 5}
[[3. 5. 4. 0. 0. 0.]
 [3. 0. 0. 2. 5. 0.]
 [0. 5. 0. 2. 0. 5.]
 [0. 0. 4. 0. 5. 5.]]


In [16]:
test_capacities = np.fromiter(nx.get_node_attributes(test, "port_capacity").values(), dtype=float)
test_priorities = test_A

#Same problem as above: unsure how to get actual transfer numbers out of this
wfq_X = weighted_fair_queueing(test_capacities, test_priorities)
wfq = pd.DataFrame(data=wfq_X)
wfq

,0,1,2,3
0,18.556701,30.927835,24.742268,25.773196
1,117.647059,101.960784,101.960784,78.431373
2,108.411215,130.841121,108.411215,52.336449
3,50.000000,50.000000,50.000000,50.000000


In [17]:
print(sum(wfq_X[0]))
print(sum(wfq_X[1]))
print(sum(wfq_X[2]))
print(sum(wfq_X[3]))

100.00000000000001
400.00000000000006
400.00000000000006
200.0000000000001


This is an alternative implementation of weighted fair queueing. It works, but way underallocates bandwidth.

In [18]:
# Priority matrix in this case has each [row, col] represent a node
# Ex: [1,2] represents an edge between node 1 and node 2
def weighted_fair_queuing_2(node_capacities, priority_matrix):
    num_nodes = len(node_capacities)
    allocation_matrix = np.zeros((num_nodes, num_nodes))
    remaining_capacities = node_capacities.copy()
    
    # Normalize priorities to make them proportional
    normalized_priorities = np.where(priority_matrix > 0, priority_matrix, 1)
    normalized_priorities_sum = normalized_priorities.sum(axis=1, keepdims=True)
    normalized_priorities = np.where(normalized_priorities_sum > 0, normalized_priorities / normalized_priorities_sum, 0)
    
    # Store handled edges so we don't repeat
    handled_edges = set()

    # Allocate bandwidth based on priorities
    for i in range(num_nodes):
        for j in range(num_nodes):
            # Only consider half the matrix (as other half is repeats)
            if i < j and priority_matrix[i, j] > 0:
                # Calculate available bandwidth at node i and j
                available_bandwidth_i = remaining_capacities.get(i)
                available_bandwidth_j = remaining_capacities.get(j)
                
                # Calculate the maximum possible allocation for the edge
                max_allocation = min(available_bandwidth_i, available_bandwidth_j)
                
                # Calculate allocated bandwidth based on priority
                allocated_bandwidth = max_allocation * normalized_priorities[i, j]
                
                # Ensure allocated bandwidth does not exceed the capacity of either node
                allocation = min(allocated_bandwidth, available_bandwidth_i, available_bandwidth_j)
                allocation_matrix[i, j] = allocation
                
                # Update remaining capacities
                remaining_capacities[i] -= allocation
                remaining_capacities[j] -= allocation
                
                # Mark edge (i, j) and (j, i) as handled
                handled_edges.add((i, j))
                handled_edges.add((j, i))
    
    # Generate list of bandwidth assignments
    edge_bandwidth_list = [
        (i, j, allocation_matrix[i, j])
        for i in range(num_nodes)
        for j in range(num_nodes)
        if i < j and allocation_matrix[i, j] > 0
    ]
    
    return allocation_matrix, edge_bandwidth_list

In [19]:
# Testing weighted_fair_queuing_2
node_capacities = {0: 100, 1: 400, 2: 500}
priority_matrix = np.array([
    [0, 1, 2],
    [1, 0, 0],
    [2, 0, 0]
])

allocation_matrix, bandwidth = weighted_fair_queuing_2(node_capacities, priority_matrix)
print("Allocation Matrix:")
print(allocation_matrix)
print(f"Bandwidth: {bandwidth}")

Allocation Matrix:
[[ 0.  25.  37.5]
 [ 0.   0.   0. ]
 [ 0.   0.   0. ]]
Bandwidth: [(0, 1, 25.0), (0, 2, 37.5)]


This is my attempt at weighted fair queuing with linear programming. Like with my first attempt at weighted fair queuing, it does assign bandwidth to each transfer, but I don't know how to interpret the results of the matrix. Also, bandwidth assigned exceeds capacity of certain nodes. 

In [20]:
def weighted_fair_queueing_linprog(capacities, priorities):
    # m is the number of source nodes, n is the number of destination nodes
    m, n = priorities.shape  
    
    # Initialize the objective function coefficients (to minimize the negative sum of allocations)
    # To maximize the priorities, you can minimize negative priorities
    # So we turn the 2D matrix into a 1D vector and negate it
    c = -priorities.flatten()

    # Setup inequality constraints based on each node's capacity
    A_ineq = []
    for i in range(m):
        capacity_constraint = np.zeros(m * n)
        for j in range(n):
            capacity_constraint[i * n + j] = 1
        A_ineq.append(capacity_constraint)
    
    A_ineq = np.array(A_ineq)
    b_ineq = capacities  

    # Setup equality constraints for the relative priorities
    # Ex: Node A has edge (A,B) with priority 5, and edge (A,D) with priority 3. 
    # Ratio is preserved in equality in the form of [5 3 0 0 ...] and appended to list
    # This is equivalent to 5x + 3y = 0, meaning capacity should be full
    A_eq = []
    b_eq = []
    for i in range(m):
        for j in range(n):
            for k in range(n):
                if j != k and priorities[i][j] > 0 and priorities[i][k] > 0:
                    eq_constraint = np.zeros(m * n)
                    eq_constraint[i * n + j] = priorities[i][k]
                    eq_constraint[i * n + k] = -priorities[i][j]
                    A_eq.append(eq_constraint)
                    #Creates vector of zeroes so that the equation in A can be equal to 0
                    b_eq.append(0)
    
    A_eq = np.array(A_eq)
    b_eq = np.array(b_eq)
    
    # Solve the linear program
    result = linprog(c, A_ub=A_ineq, b_ub=b_ineq, A_eq=A_eq, b_eq=b_eq, method='highs')
    
    if result.success:
        X_vector = result.x
        X = X_vector.reshape((m, n))
        return X
    else:
        raise ValueError("Linear programming failed to find a solution")

In [21]:
# Problem: Bandwidth is perfect across rows, but across columns (the transferes)
# Maximum bandwidth either exceeds capacity or will under utilize it
X = weighted_fair_queueing_linprog(test_capacities, test_priorities)
display_allocation = pd.DataFrame(data=X)
display_allocation

,0,1,2,3,4,5
0,25.0,41.666667,33.333333,0.000000,0.000000,0.000000
1,120.0,0.000000,0.000000,80.000000,200.000000,0.000000
2,0.0,166.666667,0.000000,66.666667,0.000000,166.666667
3,0.0,0.000000,57.142857,0.000000,71.428571,71.428571


In [22]:
for r in range(display_allocation.shape[0]):
    print(display_allocation.iloc[r].sum())

100.0
400.0
400.0
199.99999999999997


In [23]:
test_priorities

array([[3., 5., 4., 0., 0., 0.],
       [3., 0., 0., 2., 5., 0.],
       [0., 5., 0., 2., 0., 5.],
       [0., 0., 4., 0., 5., 5.]])

In [24]:
test_nodes

['A', 'B', 'C', 'D']

This is my implementation of max_min_fairness without demand. While it can allocate bandwidth for the first transfer, it exceeds node capacities when other transfers are considered. A constraint could be considered, but I'm not sure it will work. 

In [25]:
c = nx.Graph()
c.add_node("A", port_capacity=100)
c.add_node("B", port_capacity=400)
c.add_node("C", port_capacity=50)

c.add_edge("A", "B", priority=2)
c.add_edge("A", "C", priority=4)

# Create a list of nodes and a node index mapping
c_nodes = list(c.nodes)
c_node_index = {node: i for i, node in enumerate(c_nodes)}
print(c_node_index)

# List the edges and assign an index to each
c_edges = list(c.edges(data=True))
c_edge_index = {tuple(edge[:2]): i for i, edge in enumerate(c_edges)}
print(c_edge_index)

# Initialize the incidence matrix
c_n_nodes = len(c_nodes)
c_n_edges = len(c_edges)
test_C = np.zeros((c_n_nodes, c_n_edges))

# Populate the incidence matrix
for edge, index in c_edge_index.items():
    u, v = edge
    priority = c[u][v]['priority']  # Access the 'priority' attribute correctly
    i = c_node_index[u]
    j = c_node_index[v]
    
    test_C[i, index] = priority  # Outgoing edge from node i
    test_C[j, index] = priority  # Incoming edge to node j

print(test_C)

{'A': 0, 'B': 1, 'C': 2}
{('A', 'B'): 0, ('A', 'C'): 1}
[[2. 4.]
 [2. 0.]
 [0. 4.]]


In [26]:
def max_min_fairness_priorities(capacities, priorities, node_index, edge_index):    
    #normalize each row in priorities
    normalized_priorities = []
    for p in priorities:
        row_sum = sum(p)
        normalized_priorities.append(p / row_sum)

    temp_assignment = []

    #Assign minimum bandwidth to each transfer
    for r in range(len(normalized_priorities)):
        current = capacities[r] * normalized_priorities[r]
        temp_assignment.append(current)

    #For each column in temp_assignment, find the minimum bandwidth
    #Then sum those minimum bandwidths and see how much capacity each node has left
    transfer_dict = {}
    for c in range(len(temp_assignment[0])):
        col = [row[c] for row in temp_assignment if row[c]!=0]
        min_bandwidth = min(col)
        transfer_dict.update({c:min_bandwidth})

    #Recreate priority matrix with new allocations
    min_priorities = np.zeros((len(node_index), len(edge_index)))

    for edge, index in edge_index.items():
        u, v = edge
        priority = transfer_dict.get(index)  # Get priority from transfer_dict
        i = node_index[u]
        j = node_index[v]
        
        min_priorities[i, index] = priority  # Outgoing edge from node i
        min_priorities[j, index] = priority  # Incoming edge to node j
    
    #Find how much bandwidth can be distributed
    for r in range(len(min_priorities)):
        remaining_capacity = capacities[r] - sum(min_priorities[r])
        if remaining_capacity > 0:
            # Distribute remaining capacity fairly to transfers with higher priorities
            for c in range(len(normalized_priorities[r])):
                if remaining_capacity <= 0:
                    break  # Stop if no capacity remains
                
                # Calculate extra allocation proportional to the priority
                extra_bandwidth = remaining_capacity * normalized_priorities[r][c]
                min_priorities[r][c] += extra_bandwidth
                remaining_capacity -= extra_bandwidth
    
    return min_priorities

In [27]:
# From here, confused as to how we can get the bandwidth needed for each transfer
X = max_min_fairness_priorities([100,400,50], test_C, c_node_index, c_edge_index)
norm_priority = pd.DataFrame(data=X)
norm_priority

,0,1
0,38.888889,57.407407
1,400.000000,0.000000
2,0.000000,50.000000


I attempted to calculate total bandwidth assigned per transfer by summing the rows in the resulting matrix, but the results were either under capacity or way over-exceeded capacity.

In [28]:
#Doesn't sum up to max capacity exactly
for r in X:
    print(f"Sum for transfer: {sum(r)}")

# Assign bandwidth based on the minimum bandwidth for a transfer
# as calculated by weighted fair queueing
assignments = []
for c in range(len(X[0])):
    col = [row[c] for row in X if row[c]!=0]
    min_bandwidth = min(col)
    assignments.append(min_bandwidth)
print(f"Total bandwidth used by transfers: {sum(assignments)}")

Sum for transfer: 96.2962962962963
Sum for transfer: 400.0
Sum for transfer: 50.0
Total bandwidth used by transfers: 88.88888888888889


In [29]:
test_priorities

array([[3., 5., 4., 0., 0., 0.],
       [3., 0., 0., 2., 5., 0.],
       [0., 5., 0., 2., 0., 5.],
       [0., 0., 4., 0., 5., 5.]])